# Restricted Boltzman Machine based topic modelling

A restricted Boltzmann Machine is an "Energy Based" generative stochastic model. They were initially invented by Paul Smolensky in 1986 and were called "Harmonium". After the evolution of training algorithms in the mid 2000's by Geoffrey Hinton, the boltzman machine became more prominent. They gained big popularity in recent years in the context of the Netflix Prize where RBMs achieved state of the art performance in collaborative filtering and have beaten most of the competition.

RBM's are useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling. 

In this project, we use RBMs for topic modelling.

## Architecture
RBMs are shallow, two-layer neural nets that constitute the building blocks of deep-belief networks. The first layer of the RBM is called the visible, or input, layer, and the second is the hidden layer. The absense of an output layer is apparent. 
<img src="figure1.png" width="150" height="50" title="Layers in RBM">
Figure1: Layers in RBM
Each circle in the figure above represents a neuron-like unit called a node, and nodes are simply where calculations take place. 
<img src="figure3.png" width="500" height="200" title="Layers in RBM">
Figure2: Structure of RBM
The nodes are connected to each other across layers, but no two nodes of the same layer are linked. That is, there is no intra-layer communication – this is the restriction in a restricted Boltzmann machine. Each node is a locus of computation that processes input, and begins by making stochastic decisions about whether to transmit that input or not. Each visible node takes a low-level feature from an item in the dataset to be learned.

Let's start by importing the packages that will be required in this project.

In [1]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
np_rng = np.random.RandomState(1234) #setting the random state

In [ ]:
df = pd.read_excel("~/library/amazon.xlsx")

In [3]:
df.head()

,Text
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In Topic Modelling, we find the best set of topics that describe the document. There are various ways to perform topic modelling one of which is RBM. We can train an RBM on a set of documents. 
The visible layers will be the words in the text, the hidden layers will give the Topics. 
To input words into the visible layer, We'll convert the train and test data that was split above into a bag of words model.

In [8]:
# creating bag of words model on train and test data

tf = CountVectorizer(stop_words='english')

# fit tf on the dataframe df
tf = tf.fit(df['Text'])

# transform df dataframe
trainX = tf.transform(df['Text'])

Defining the number of visible and hidden units.

In [17]:
# define visible units
visibleUnits = trainX.shape[1]

# assign number of units
hiddenUnits = 5 

In [18]:
visibleUnits

1642

In [20]:
# utility Functions

# deine the sigmoid function
def sigmoid(X):
    return 1/(1+np.exp(-X))

## RBM as a Probabilistic Model
Restricted Boltzmann Machines are probabilistic. As opposed to assigning discrete values the model assigns probabilities. At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers v and h. The probability that a certain state of v and h can be observed is given by the following joint distribution:
<img src="figure4.png" width="200" height="70" title="Layers in RBM">

Eq. 2. Joint Distribution for v and h.
Here Z is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

The joint distribution is known as the Boltzmann Distribution which gives the probability that a particle can be observed in the state with the energy E. Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of v and h in the partition function Z. Much easier is the calculation of the conditional probabilities of state h given the state v and conditional probabilities of state v given the state h:
<img src="figure5.png" width="200" height="70" title="Layers in RBM">

Eq. 3. Conditional probabilities for h and v.
It should be noticed beforehand (before demonstrating this fact on practical example) that each neuron in a RBM can only exist in a binary state of 0 or 1. The most interesting factor is the probability that a hidden or visible layer neuron is in the state 1 — hence activated. 

# Contrastive Divergence

### Gibbs Sampling
The first part of the training is called Gibbs Sampling. Given an input vector v we are using p(h|v) for prediction of the hidden values h via sampling. Knowing the hidden values we use p(v|h) for prediction of new input values v via sampling. This process is repeated k times. After k iterations, we obtain the visible vector $v_k$ which was recreated from original input values $v_0$.
<img src="figure8.png" width="500" height="300" title="Layers in RBM">

The gibbs function *gibbs* is divided into subparts: <br>
1.*sampleHiddenLayer * <br>
2.*sampleVisibleLayer*

### Sample Hidden Layer

As already stated, that given an input vector v the probability for a single hidden neuron j being activated is:
<img src="figure6.png" width="400" height="200" title="Layers in RBM">

Eq. 4
Here is σ the Sigmoid function.

*sampleHiddenLayer* takes the visible layer as input to calculate the hidden layer using Eq. 4 *h1Pdf* and then samples it to get * h1_sample*

    v_sample: given visible layer matrix; matrix because a batch of data points will be trained at one go
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    hPdf: distribution of hidden layer; a matrix for batch of datapoints = p(h|v)
    h_sample: sampled hidden layer matrix

In [38]:
def sampleHiddenLayer(v_sample):
    
    #code for calculation of hPdf using vectorized implementation of Eq 4
    hPdf = sigmoid(hiddenBias + np.dot[np.transpose(W), v_sample])# ~ 1 line
    
    # Here, np.random.binomial is used to create the hidden layer sample matrix
    h_sample = np_rng.binomial(size=hPdf.shape, n=1, p=hPdf)
    return [hPdf, h_sample]

### Sample Visible Layer
Similarly, the probability that a binary state of a visible neuron i is set to 1 is:
<img src="figure7.png" width="400" height="200" title="Layers in RBM">
Eq. 5

As seen in equation 5, we will be writing a function to sample the Visible Layer.
This function samples the visible layer based on the sampled data of hidden layer. <br>

There are some differences in writing the function *sampleVisibleLayer*. <br>Firstly, we use np.random.multinomial to sample the visible layer *v_sample* from the distribution *vPdf*. <br>Secondly,elements of *vPdf* needs to sum to 1 as the function np.random.multinomial used to sample the visible layer takes on probability distributions as *pvals*. In other words, we are finding the softmax values. <br> Thirdly, we also make use of the *D* to sample the visible layer as each document has different word count.
    
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    returns a sample vector of hidden layer and its distribution for a batch of data points
    
    vPdf: distribution of visible layer; a matrix for batch of datapoints = p(v|h)
    v_sample: sampled visible layer matrix
    

In [24]:
def sampleVisibleLayer(h_sample, D):
    
    
    numerator = sigmoid(visibleBias + np.dot(W,h_sample))
    denominator = numerator.sum(axis=1)
    vPdf = numerator/denominator
    
    # Here np.random.multinomial is used to sample as each document has different number of words 
    # and hence D is also a parameter in sampling
    v_sample = np.zeros((batchSize, vPdf.shape[1]))
    for i in range(batchSize):
        v_sample[i] = np_rng.multinomial(size=1, n=D[i], pvals=vPdf[i])
    return [vPdf, v_sample]

### Sampling

We use the above functions to write the function *gibbs* to run one iteration of gibbs sampling. We are calculating the visible layer samples first and then using it to calculate he hidden layer sample. 
    
    Input:
    h_sample: given hidden layer matrix; matrix because a batch of data points will be trained at one go
    D: array of the sum of the row of the data vector; vector containing number of words in each document
    
    Output:
    vPdf: distribution of visible layer
    v_sample: sampled visible layer matrix
    hPdf: distribution of hidden layer
    h_sample: sampled hidden layer matrix

In [25]:
def gibbs(h_sample, D):
    
    #use sampleVIsibleLayer and sampleHiddenLayer 
    vPdf, v_sample = sampleVisibleLayer(h_sample, D)[0], sampleVisibleLayer(h_sample, D)[1]# ~1 line
    hPdf, h_sample = sampleHiddenLayer(v_sample)# ~1 line
    return [vPdf, v_sample, hPdf, h_sample ]


### Contrastive Divergence

Contrastive Divergence updates weights after one iteration of Gibbs Sampling. Here, we shall perform *k* such iterations in Contrastive Divergence. 
The update of the weight matrix happens post the Contrastive Divergence step. 


Now, we will be writing a funtion to run the contrastive divergence algorithm in k steps
    
    Input:
    data: batch data (visible layer)
    k: no of iterations for gibbs sampling
    
    Output:
    hiddenPDF_data: distribution of the hidden layer based on data
    visibleSamples: visible samples generated by gibbs sampling 
    hiddenPDF: distribution of the hidden layer based on samples generated by gibbs

In [26]:
def cd_k(data,k):
    
    D = data.sum(axis=1)
    hiddenPDF_data, hiddenSample_data = sampleHiddenLayer(data)
    chain_start = hiddenSample_data

    for step in range(k):
        if step == 0:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples  = gibbs(chain_start, D)
        else:
            visiblePDF, visibleSamples, hiddenPDF, hiddenSamples = gibbs(hiddenSamples)
    return hiddenPDF_data, visibleSamples, hiddenPDF


## Training the Model

Vectors $v_0$ and $v_k$ are used to calculate the activation probabilities for hidden layers $p(h_0|v_0)$ and $p(h_k|v_k)$ (Eq.4). The difference between the outer products of those probabilities with input vectors $v_0$ and $v_k$ results in the update matrix:
<img src="figure9.png" width="300" height="200" title="Layers in RBM">
Eq. 6. Update matrix. 

In order to calculate $\Delta (bias)$, <br>
<center>$\Delta (visiblebias) = average\_ across\_ batch(v_0 - v_k)$ </center> 
<center>$\Delta (hiddenbias) = average\_across\_ batch(p(h_0|v_0) - p(h_k|v_k))$ </center> 

Using the update matrix the new weights can be calculated with momentum gradient ascent, given by:
<center>  $mW_t = \gamma \ mW_{t-1} - \Delta W$</center> 
<center>  $mvisiblebias_t = \gamma \ mvisiblebias_{t-1} - \Delta visiblebias$</center>
<center>  $mhiddenbias_t = \gamma \ mhiddenbias_{t-1} - \Delta hiddenbias$</center><br>
<center>  $W_t = W_{t-1} + \alpha \ mW_t$</center> 
<center>  $visiblebias_t = visiblebias_{t-1} + \alpha \ mvisiblebias_t$</center> 
<center>  $hiddenbias_t = hiddenbias_{t-1} + \alpha \ mhiddenbias_t$</center> 


Eq. 7. Update rule for the weights.

Note that in the code implementation below <br>
 hiddenPDF_data is $p(h_0|v_0)$ <br>
 visibleSamples is $v_k$ <br>
 hiddenPDF is $p(h_k|v_k)$ <br>
 mdata is $v_0$ <br>
 eta is $\alpha$ <br>
 mrate is $\gamma$ <br>


In this we will write a function which iterates over our data for *epochs*.
At every epoch we shuffle the data and then run CD on a mini batch size defined by *batchSize*

### Initializations

In [27]:
"""
visibleUnits: no of words in your Bag of words Model
hiddenUnits: no of topics
batchSize: data slice to be selected 
epochs: no of iterations
eta: learning rate
mrate: momentum coefficient
W : weights between the visible and hidden layer
visibleBias, hiddenBias: biases for visible and hidden layer respectively
"""

# define batch size
batchSize = 200

epochs = 100
eta = 0.05 
mrate = 0.5

# initialise weights
weightinit=0.01
W = weightinit * np_rng.randn(visibleUnits, hiddenUnits)
visibleBias = weightinit * np_rng.randn(visibleUnits)
hiddenBias = np.zeros((hiddenUnits))

# we shall use 10 interations of gibbs sampling
k=10

### Training

In [28]:
def train(dataX,k):
    
    mW = np.zeros((visibleUnits, hiddenUnits))
    mvisibleBias =  np.zeros(visibleUnits)
    mhiddenBias =  np.zeros(hiddenUnits)
    global W,visibleBias,hiddenBias,mrate,batchSize,epochs 
    for epoch in range(epochs):
        np_rng.shuffle(dataX) #shuffling the data
        
        for i in range(0, dataX.shape[0], batchSize):
            mData = dataX[i:i+batchSize]
            hiddenPDF_data, visibleSamples, hiddenPDF =  cd_k(mData,k)# perfroming Contrastive Divergence on the batch for k iterations

            mW = mrate*mW - (np.outer(visibleSamples[0], hiddenPDF_data)-np.outer(visibleSamples[-1], hiddenPDF)) #writing the momentum update equation for weight matrix
            mvisibleBias =  mrate*mvisibleBias - np.mean(visibleSamples[0]-visibleSamples[-1])#writing the momentum update equation for visiblebias vector
            mhiddenBias =  mrate*mhiddenBias - np.mean(hiddenPDF_data - hiddenPDF)#writing the momentum update equation for hiddenbias vector

            W = W + eta*mW #weight update equation
            visibleBias = visibleBias + eta*mvisibleBias  #visible bias update equation
            hiddenBias =  hiddenBias + eta*mhiddenBias#hidden bias update equation

## Train the Model



In [39]:
train(trainX.toarray(),k)

TypeError: 'builtin_function_or_method' object is not subscriptable

# Word Distribution based on Topics
In this function, we are finding the distribution of words over the topics. The number of topics is the number of neurons in the hidden layer. <br>
<br>

For each topic, the function prints the top 15 words that describe the topic. It can be seen that some of the words occur in multiple topics.

    topic: number of hidden layers
    voc: indexing of the vocabulary

In [32]:
def worddist( topic, voc):
    
    """
    Initialize every topic =1 once 
    """
    vecTopics = np.zeros((topic, topic))
    for i in range(len(vecTopics)):
        vecTopics[i][i] = 1
    
    
    for i, vecTopic in enumerate(vecTopics):
       
        numerator = np.exp(np.dot(vecTopic, W.T) + visibleBias)
        denominator = numerator.sum().reshape((1, 1))
        word_distribution = (numerator/denominator).flatten()
        
        tmpDict = {}
        for j in voc.keys():
            tmpDict[j] = word_distribution[voc[j]]
        print('topic', str(i), ':', vecTopic)
        lm=0
        for word, prob in sorted(tmpDict.items(), key=lambda x:x[1], reverse=True):
            print ( word, str(prob))
            lm+=1
            if lm==15:
                break
        print('\n')

In [33]:
worddist( hiddenUnits, tf.vocabulary_)

topic 0 : [1. 0. 0. 0. 0.]
spinn 0.0006379565177117481
scratched 0.000636152382777895
controls 0.0006334030394393014
loose 0.000632183301117277
50 0.0006321175296802414
noted 0.000631780973983011
somewhat 0.0006316763325665434
gadgets 0.0006315498283358207
ears 0.0006314718932895621
regretted 0.0006310663695762577
speakerphone 0.0006307949173899025
numbers 0.0006303221910363814
purpose 0.0006298770156734586
dissapointed 0.0006294847807677973
free 0.0006292137566182452


topic 1 : [0. 1. 0. 0. 0.]
echo 0.000644964663961509
speakerphone 0.0006354094770505684
noted 0.000633974509663662
died 0.0006333647269084327
music 0.0006312707795136227
controls 0.0006306776792956029
e715 0.0006306574750786232
absolutel 0.0006305492952554221
awful 0.0006303984536323288
looking 0.0006302364971139969
flawlessly 0.0006301742043026613
overly 0.0006292417618318936
inexpensive 0.0006290120676221188
lesson 0.0006289851352568824
shiny 0.0006282460140366275


topic 2 : [0. 0. 1. 0. 0.]
looking 0.000635154246945

The output shows the probability assigned to each word for ever topic present.